In [1]:
import sys
print(sys.version)

3.10.11 (v3.10.11:7d4cc5aa85, Apr  4 2023, 19:05:19) [Clang 13.0.0 (clang-1300.0.29.30)]


This notebook measures and compares the runtime for (1) the main upper bound algorithm, (2) constructing $\hat{\Delta}$, and (3) computing all pairwise distances from the feature vectors. 

In [2]:
import numpy as np
import time
import pandas as pd
from pathlib import Path
from silhouette_upper_bound import upper_bound
from sklearn.metrics import pairwise_distances
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
from tqdm import tqdm

In [3]:
# ============================================================
# CONFIGURATION
# ============================================================

BASE_PATH = Path.cwd()

ALGO_OUTPUT_PATH = f"{BASE_PATH}/results/runtime_results.csv"
D_HAT_OUTPUT_PATH = f"{BASE_PATH}/results/runtime_results_D_hat.csv"
D_OUTPUT_PATH = f"{BASE_PATH}/results/runtime_results_distance_matrix.csv"

# Dataset sizes to test
#SIZES = [2000, 5000, 10000, 20000, 30000, 40000]  
SIZES = [2000, 5000, 10000]  

# Dimension of synthetic points
DIM = 32

# Number of repeats for averaging runtimes
REPEATS = 5

In [4]:
# ============================================================
# RUNTIME MEASUREMENT
# ============================================================
def measure_runtime(func, func_input, repeats=5):
    runtimes = []
    for _ in range(repeats):
        start = time.perf_counter()
        func(func_input)
        end = time.perf_counter()
        runtimes.append(end - start)

    return np.mean(runtimes), np.std(runtimes)

In [5]:
# ============================================================
# MAIN LOOP
# ============================================================
def run_experiments(output_file):
    results = []

    pbar = tqdm(SIZES)

    for n in pbar:
        pbar.set_description(f"Number of samples (n): {n}")

        # Synthetic Gaussian data
        np.random.seed(0) 
        X = np.random.randn(n, DIM).astype(np.float32) # feature vectors
        D = pairwise_distances(X, metric="euclidean") # distance matrix 

        # Algo
        if output_file == ALGO_OUTPUT_PATH:
            mean_rt, std_rt = measure_runtime(upper_bound, D, REPEATS)

        # Constructing D_hat
        elif output_file == D_HAT_OUTPUT_PATH:
            mean_rt, std_rt = measure_runtime(lambda D: np.sort(D[~np.eye(D.shape[0], dtype=bool)].reshape(D.shape[0], -1)), D, REPEATS)
        
        # Constructing D
        elif output_file == D_OUTPUT_PATH:
            mean_rt, std_rt = measure_runtime(pairwise_distances, X, REPEATS)

        results.append({
            "n_samples": n,
            "dim": DIM,
            "repeats": REPEATS,
            "runtime_mean_sec": mean_rt,
            "runtime_std_sec": std_rt
        })

    df = pd.DataFrame(results)
    df.to_csv(output_file, index=False)

    return df

In [6]:
run_experiments(ALGO_OUTPUT_PATH)

Number of samples (n): 10000: 100%|██████████| 3/3 [00:21<00:00,  7.02s/it]


,n_samples,dim,repeats,runtime_mean_sec,runtime_std_sec
0,2000,32,5,0.137864,0.126560
1,5000,32,5,0.782837,0.021938
2,10000,32,5,3.128312,0.113279


In [7]:
run_experiments(D_HAT_OUTPUT_PATH)

Number of samples (n): 10000: 100%|██████████| 3/3 [00:09<00:00,  3.21s/it]


,n_samples,dim,repeats,runtime_mean_sec,runtime_std_sec
0,2000,32,5,0.063457,0.014568
1,5000,32,5,0.326449,0.032725
2,10000,32,5,1.383796,0.030097


In [8]:
run_experiments(D_OUTPUT_PATH)

Number of samples (n): 10000: 100%|██████████| 3/3 [00:04<00:00,  1.54s/it]


,n_samples,dim,repeats,runtime_mean_sec,runtime_std_sec
0,2000,32,5,0.027794,0.003232
1,5000,32,5,0.138152,0.022703
2,10000,32,5,0.579185,0.045095


In [9]:
# Generate figure 

plt.style.use("seaborn-v0_8-whitegrid")

df = pd.read_csv(ALGO_OUTPUT_PATH)
df_distance_matrix = pd.read_csv(D_OUTPUT_PATH)
df_D_hat = pd.read_csv(D_HAT_OUTPUT_PATH) 

fig, ax = plt.subplots(figsize=(9, 6))

colors = ['#000000', '#969696', '#6baed6', '#31a354']
markers = ['o', '', 'x']
linestyles = ['-', '-', '--']
linewidths = [1.5, 1.5, 1.5]

# --- 1. Runtime of Algorithm 1 ---
ax.errorbar(
    df["n_samples"], 
    df["runtime_mean_sec"], 
    yerr=df["runtime_std_sec"], 
    marker=markers[0], 
    linestyle=linestyles[0], 
    color=colors[0], 
    linewidth=linewidths[0], 
    markersize=7, 
    capsize=3, 
    label='Algorithm 1'
)

# --- 2. Runtime of constructing D_hat ---
ax.errorbar(
    df_D_hat["n_samples"], 
    df_D_hat["runtime_mean_sec"], 
    yerr=df_D_hat["runtime_std_sec"], 
    marker='o',              
    markerfacecolor='none',
    linestyle=linestyles[1], 
    color=colors[1], 
    linewidth=linewidths[1], 
    markersize=7, 
    capsize=3,
    label='Constructing $\hat{\Delta}$ (part of Algorithm 1)'
)

# --- 3. Reference: distance matrix computation (Dissimilarity Matrix) ---
ax.errorbar(
    df_distance_matrix["n_samples"], 
    df_distance_matrix["runtime_mean_sec"], 
    yerr=df_distance_matrix["runtime_std_sec"], 
    marker=markers[2], 
    linestyle=linestyles[2], 
    color=colors[2], 
    linewidth=linewidths[2], 
    markersize=7, 
    capsize=3,
    label='Computing dissimilarity matrix $\Delta$'
)


ax.set_xlabel("Number of samples ($n$)", fontsize=14)
ax.set_ylabel("Runtime (seconds)", fontsize=14)

ax.set_yscale('log')

ax.legend(loc='upper left', frameon=True, shadow=False, borderpad=0.8, fontsize=14, handlelength=3)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')

# Use ScalarFormatter to display actual numbers (10, 100) instead of scientific notation (1e1, 1e2)
ax.xaxis.set_major_formatter(ScalarFormatter())

form = plt.FormatStrFormatter('%g')
plt.gca().yaxis.set_major_formatter(form)

#Force all X ticks to display, matching previous request
ax.set_xticks(df["n_samples"].values) 
ax.tick_params(axis='x', rotation=45, which='major', labelsize=14)
ax.tick_params(axis='y', which='major', labelsize=14)

plt.tight_layout()
plt.savefig(f"{BASE_PATH}/figures/runtime.pdf", bbox_inches="tight", dpi=300)
plt.close()